<a href="https://colab.research.google.com/github/Rogerio-mack/IMT_CD_2025/blob/main/IMT_ex_knn_dt_solucao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<head>
  <meta name="author" content="Rogério de Oliveira">
  <meta institution="author" content="ITM">
</head>

<img src="https://maua.br/images/selo-60-anos-maua.svg" width=300, align="right">
<!-- <h1 align=left><font size = 6, style="color:rgb(200,0,0)"> optional title </font></h1> -->




# **Exercício 3 - 3o bimestre - ECM514 Ciência de Dados**

In [ ]:
#@markdown Nome completo e RA (*\<nome\>, \<RA\>*)
Aluno = 'Daniel Henrique, 1115665' #@param {type:"string"}






# **Classificação: Knn $\times$ Árvores de Decisão**
---



# Caso: **Predição de Diagnósticos a partir de Dados de Imagens**


https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+(Diagnostic)

Os dados acima são características computadas a partir de uma imagem digitalizada por *agulha fina* (PAAF) de uma massa mamária. Eles descrevem características dos núcleos celulares presentes na imagem e classificam os tumores como malignos ou benignos.







In [ ]:
#@markdown **Set Up Lab** (*run this before continue*)

import pandas                  as pd
import numpy                   as np
import matplotlib.pyplot       as plt
import warnings
import os
import seaborn as sns
warnings.filterwarnings("ignore")

breast = pd.read_csv('https://github.com/Rogerio-mack/Machine-Learning-I/raw/main/data/breast.csv')

from sklearn.model_selection import train_test_split

seed = 1984
X_train, X_test, y_train, y_test = train_test_split(breast.drop(columns=['diagnosis']), breast.diagnosis, test_size=0.05, stratify=breast.diagnosis, random_state=seed)

breast = pd.concat([X_train, y_train],axis=1)
new_breast = X_test

breast.to_csv('breast.csv',index=False)

new_breast['diagnosis'] = '?'
new_breast.to_csv('new_breast.csv',index=False)

y_test.to_csv('new_breast_answers.csv',index=False)

# correlation_matrix = breast.drop(columns=['diagnosis']).corr()

# Find pairs with correlation > 0.99, excluding the diagonal
# highly_correlated_pairs = correlation_matrix[abs(correlation_matrix) > 0.99].stack()
# highly_correlated_pairs = highly_correlated_pairs[highly_correlated_pairs != 1.0]

# print("Highly correlated pairs (correlation > 0.99):")
# display(highly_correlated_pairs)


E você empregará os dados já tratados no set up acima, com os dataframes `breast` e `new_breast`. A idea aqui é empregar os dados de `breast` para estimar as classes dos novos casos desconhecidos de `new_breast`.



In [ ]:
breast.head()

,id,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,diagnosis
411,8911670,18.810,19.98,120.90,1102.0,0.08923,0.05884,0.08020,0.05843,0.1550,...,24.30,129.00,1236.0,0.12430,0.1160,0.2210,0.12940,0.2567,0.05737,M
149,9047,12.940,16.17,83.18,507.6,0.09879,0.08836,0.03296,0.02390,0.1735,...,23.02,89.69,580.9,0.11720,0.1958,0.1810,0.08388,0.3297,0.07834,B
103,859196,9.173,13.86,59.20,260.9,0.07721,0.08751,0.05988,0.02180,0.2341,...,19.23,65.59,310.1,0.09836,0.1678,0.1397,0.05087,0.3282,0.08490,B
260,864496,8.726,15.83,55.84,230.9,0.11500,0.08201,0.04132,0.01924,0.1649,...,19.62,64.48,284.4,0.17240,0.2364,0.2456,0.10500,0.2926,0.10170,B
106,91505,12.540,16.32,81.25,476.3,0.11580,0.10850,0.05928,0.03279,0.1943,...,21.40,86.67,552.0,0.15800,0.1751,0.1889,0.08411,0.3155,0.07538,B


In [ ]:
new_breast.head()

,id,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,diagnosis
274,862009,13.45,18.30,86.60,555.1,0.10220,0.08165,0.03974,0.02780,0.1638,...,25.94,97.59,699.4,0.13390,0.17510,0.13810,0.07911,0.2678,0.06603,?
107,9110127,18.03,16.85,117.50,990.0,0.08947,0.12320,0.10900,0.06254,0.1720,...,22.02,133.30,1292.0,0.12630,0.26660,0.42900,0.15350,0.2842,0.08225,?
264,90769601,11.13,16.62,70.47,381.1,0.08151,0.03834,0.01369,0.01370,0.1511,...,20.29,74.35,421.1,0.10300,0.06219,0.04580,0.04044,0.2383,0.07083,?
175,909777,10.57,18.32,66.82,340.9,0.08142,0.04462,0.01993,0.01111,0.2372,...,23.31,69.35,366.3,0.09794,0.06542,0.03986,0.02222,0.2699,0.06736,?
215,904689,12.96,18.29,84.18,525.2,0.07351,0.07899,0.04057,0.01883,0.1874,...,24.61,96.31,621.9,0.09329,0.23180,0.16040,0.06608,0.3207,0.07247,?


# Exercício. Acesse e Explore os dados

Qual o atributo classe? Quantos casos são **'B'enignos** e **'M'alignos**? Existem dados ausentes (Null ou NA)? Existem atributos **não preditores**?


In [ ]:
# Seu código aqui
breast = pd.read_csv('breast.csv')
display(breast.head())

new_breast = pd.read_csv('new_breast.csv')
display(new_breast.head())


,id,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,diagnosis
0,8911670,18.810,19.98,120.90,1102.0,0.08923,0.05884,0.08020,0.05843,0.1550,...,24.30,129.00,1236.0,0.12430,0.1160,0.2210,0.12940,0.2567,0.05737,M
1,9047,12.940,16.17,83.18,507.6,0.09879,0.08836,0.03296,0.02390,0.1735,...,23.02,89.69,580.9,0.11720,0.1958,0.1810,0.08388,0.3297,0.07834,B
2,859196,9.173,13.86,59.20,260.9,0.07721,0.08751,0.05988,0.02180,0.2341,...,19.23,65.59,310.1,0.09836,0.1678,0.1397,0.05087,0.3282,0.08490,B
3,864496,8.726,15.83,55.84,230.9,0.11500,0.08201,0.04132,0.01924,0.1649,...,19.62,64.48,284.4,0.17240,0.2364,0.2456,0.10500,0.2926,0.10170,B
4,91505,12.540,16.32,81.25,476.3,0.11580,0.10850,0.05928,0.03279,0.1943,...,21.40,86.67,552.0,0.15800,0.1751,0.1889,0.08411,0.3155,0.07538,B


,id,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,diagnosis
0,862009,13.45,18.30,86.60,555.1,0.10220,0.08165,0.03974,0.02780,0.1638,...,25.94,97.59,699.4,0.13390,0.17510,0.13810,0.07911,0.2678,0.06603,?
1,9110127,18.03,16.85,117.50,990.0,0.08947,0.12320,0.10900,0.06254,0.1720,...,22.02,133.30,1292.0,0.12630,0.26660,0.42900,0.15350,0.2842,0.08225,?
2,90769601,11.13,16.62,70.47,381.1,0.08151,0.03834,0.01369,0.01370,0.1511,...,20.29,74.35,421.1,0.10300,0.06219,0.04580,0.04044,0.2383,0.07083,?
3,909777,10.57,18.32,66.82,340.9,0.08142,0.04462,0.01993,0.01111,0.2372,...,23.31,69.35,366.3,0.09794,0.06542,0.03986,0.02222,0.2699,0.06736,?
4,904689,12.96,18.29,84.18,525.2,0.07351,0.07899,0.04057,0.01883,0.1874,...,24.61,96.31,621.9,0.09329,0.23180,0.16040,0.06608,0.3207,0.07247,?


In [ ]:
# Seu código aqui
display( breast.shape )

display( breast.isnull().sum() )

display( breast.diagnosis.value_counts() )

(513, 32)

,0
id,0
radius_mean,0
texture_mean,0
perimeter_mean,0
area_mean,0
smoothness_mean,0
compactness_mean,0
concavity_mean,0
concave points_mean,0
symmetry_mean,0


,count
diagnosis,
B,322
M,191


# Exercício. Pré-Processamento, Conjunto de Treinamento e Teste

Prepare aqui os dados de entrada `X` e saída `y` do modelo.

Note que há um atributo além do atributo classe que não é preditor (qual?). Remova esse atributo antes de normalizar os dados. Para normalização empregue o `MinMaxScaler()` do scikit-learn (ver no material de teoria exemplo de como aplicar).

Empregue a função `train_test_split` e separe 0.3 dos dados para teste. Empregue o `random_state=42` para garantir a reprodutibilidade dos resultados e empregue o parâmetro `stratify` para garantir a mesma distribuição das classes nos conjuntos de treinamento e teste.

In [ ]:
# Seu código aqui
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix

from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

In [ ]:
breast.drop(columns=['id'], inplace=True)
new_breast.drop(columns=['id'], inplace=True)

X = breast.drop(columns=['diagnosis'])
y = breast.diagnosis

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.3, random_state=42)

scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [ ]:
X_train.mean()

np.float64(0.24746078353278642)

### **Nota:**

Outras transformações como encodes, devem seguir o mesmo procedimento: aplica-se o ajusta-se e aplica-se o estimador nos dados de treinamento e, na sequência, apenas aplica-se aos dados de teste.

# Exercício. Treine modelos Knn

Treine dois modelos Knn, um com k=9 e outro com k=5. Verifique a acuracidade de cada modelo para o conjunto de teste.





In [ ]:
results = { }  # para salvar os resultados

In [ ]:
# Seu código aqui
clf = KNeighborsClassifier(n_neighbors = 9)

clf.fit(X_train,y_train)

y_pred = clf.predict(X_test)

print( y_pred[0:10], '...' )
print( f'{clf.score(X_test,y_test):.3f}' )

results[clf] = clf.score(X_test,y_test)


['M' 'B' 'B' 'B' 'M' 'M' 'M' 'B' 'B' 'M'] ...
0.961


In [ ]:
# Seu código aqui
cm = confusion_matrix(y_test, y_pred)
print('\nMatriz de Confusão:\n')
print(cm)

accuracy = accuracy_score(y_test, y_pred)
print(f'\nScore de Acuracidade (1): {accuracy:.2f}')

accuracy = clf.score(X_test, y_test)
print(f'\nScore de Acuracidade (2): {accuracy:.2f}')

print('\nClassification Report:\n')
print(classification_report(y_test, y_pred))


Matriz de Confusão:

[[94  3]
 [ 3 54]]

Score de Acuracidade (1): 0.96

Score de Acuracidade (2): 0.96

Classification Report:

              precision    recall  f1-score   support

           B       0.97      0.97      0.97        97
           M       0.95      0.95      0.95        57

    accuracy                           0.96       154
   macro avg       0.96      0.96      0.96       154
weighted avg       0.96      0.96      0.96       154



### **Nota:**

Note que há duas formas, além do `classification_report`, de se obter a acuracidade:

```
accuracy = accuracy_score(y_test, y_pred)

accuracy = clf.score(X_test, y_test)
```

O resultado é o mesmo. No segundo o modelo (no caso, `clf`) é aplicado a `X_test` para se obter os valores previsto, que são então comparados com `y_test`.


In [ ]:
# Seu código aqui
from sklearn.neighbors import KNeighborsClassifier

clf = KNeighborsClassifier(n_neighbors = 5)

clf.fit(X_train,y_train)

y_pred = clf.predict(X_test)

print( y_pred[0:10], '...' )
print( f'{clf.score(X_test,y_test):.3f}' )

results[clf] = clf.score(X_test,y_test)

['M' 'B' 'B' 'B' 'M' 'M' 'B' 'B' 'B' 'M'] ...
0.968


In [ ]:
# Seu código aqui
cm = confusion_matrix(y_test, y_pred)
print('\nMatriz de Confusão:\n')
print(cm)

print('\nClassification Report:\n')
print(classification_report(y_test, y_pred))


Matriz de Confusão:

[[95  2]
 [ 3 54]]

Classification Report:

              precision    recall  f1-score   support

           B       0.97      0.98      0.97        97
           M       0.96      0.95      0.96        57

    accuracy                           0.97       154
   macro avg       0.97      0.96      0.97       154
weighted avg       0.97      0.97      0.97       154



# Exercício. Treine modelos de Árvore de Decisão

Treine dois modelos de Árvore de Decisão. Em ambos empregue `random_state=42`. No primeiro, empregue os parâmetros padrão e, no segundo a seleção de nós por entropia e uma poda de 4 níveis da árvore.





In [ ]:
# Seu código aqui
clf = DecisionTreeClassifier(random_state=42)

clf.fit(X_train,y_train)

y_pred = clf.predict(X_test)

print( y_pred[0:10], '...' )
print( f'{clf.score(X_test,y_test):.3f}' )

results[clf] = clf.score(X_test,y_test)

['M' 'B' 'B' 'B' 'M' 'M' 'B' 'B' 'B' 'M'] ...
0.942


In [ ]:
# Seu código aqui
cm = confusion_matrix(y_test, y_pred)
print('\nMatriz de Confusão:\n')
print(cm)

print('\nClassification Report:\n')
print(classification_report(y_test, y_pred))


Matriz de Confusão:

[[93  4]
 [ 5 52]]

Classification Report:

              precision    recall  f1-score   support

           B       0.95      0.96      0.95        97
           M       0.93      0.91      0.92        57

    accuracy                           0.94       154
   macro avg       0.94      0.94      0.94       154
weighted avg       0.94      0.94      0.94       154



In [ ]:
# Seu código aqui
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier(criterion='entropy', max_depth=4, random_state=42)

clf.fit(X_train,y_train)

y_pred = clf.predict(X_test)

print( y_pred[0:10], '...' )
print( f'{clf.score(X_test,y_test):.3f}' )

results[clf] = clf.score(X_test,y_test)

['M' 'B' 'B' 'B' 'M' 'M' 'B' 'B' 'B' 'M'] ...
0.974


In [ ]:
# Seu código aqui
cm = confusion_matrix(y_test, y_pred)
print('\nMatriz de Confusão:\n')
print(cm)

print('\nClassification Report:\n')
print(classification_report(y_test, y_pred))


Matriz de Confusão:

[[95  2]
 [ 2 55]]

Classification Report:

              precision    recall  f1-score   support

           B       0.98      0.98      0.98        97
           M       0.96      0.96      0.96        57

    accuracy                           0.97       154
   macro avg       0.97      0.97      0.97       154
weighted avg       0.97      0.97      0.97       154



In [ ]:
clf.score(X_train,y_train)

0.9805013927576601

# Exercício. Prevendo novos Casos

Empregue o melhor modelo obtido (maior acuracidade) para classificar os novos casos de `new_breast`.



In [ ]:
results

{KNeighborsClassifier(n_neighbors=9): 0.961038961038961,
 KNeighborsClassifier(): 0.9675324675324676,
 DecisionTreeClassifier(random_state=42): 0.9415584415584416,
 DecisionTreeClassifier(criterion='entropy', max_depth=4, random_state=42): 0.974025974025974}

In [ ]:
best_model = max(results, key=results.get)
print(best_model)

DecisionTreeClassifier(criterion='entropy', max_depth=4, random_state=42)


In [ ]:
# Seu código aqui
X_new = new_breast.drop(columns=['diagnosis'])       # Dados para predição
X_new = scaler.transform(X_new)                      # Normalizando os dados aqui!

y_pred = best_model.predict(X_new)                   # Resposta do modelo

print(y_pred)

['B' 'M' 'B' 'B' 'B' 'M' 'B' 'M' 'M' 'B' 'M' 'B' 'M' 'M' 'M' 'B' 'M' 'B'
 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'M' 'B']


In [ ]:
# Seu código aqui
print('Bs =', sum(y_pred == 'B'))
print('Ms =', sum(y_pred == 'M'))

Bs = 17
Ms = 10


# **Nota:**

Faz sentido retreinar o melhor modelo com todos os dados disponíveis após a avaliação no conjunto de teste? **Sim**.

A ideia por trás disso é que, uma vez que você identificou o modelo com melhor desempenho nos dados de teste (que simulam dados não vistos), você pode querer que o modelo final use o máximo de informações possível para aprender os padrões nos dados.

O conjunto de teste é usado para avaliar o desempenho generalizado do modelo e evitar o *overfitting*. Depois de validar o modelo, você pode usar todo o conjunto de dados para o treinamento final, pois não precisa mais reservar uma parte para avaliação de desempenho.

Mas a diferença, sendo o conjunto X_train suficientemente representativo, pode não ser significativa.

In [ ]:
X = breast.drop(columns=['diagnosis'])
y = breast.diagnosis

scaler = MinMaxScaler()
X = scaler.fit_transform(X)

best_model = DecisionTreeClassifier(criterion='entropy', max_depth=4, random_state=42).fit(X,y)

new_breast = pd.read_csv('new_breast.csv')
new_breast.drop(columns=['id'], inplace=True)
X_new = new_breast.drop(columns=['diagnosis'])       # Dados para predição

X_new = scaler.transform(X_new)                      # Normalizando os dados aqui!

y_pred = best_model.predict(X_new)                   # Resposta do modelo

print(y_pred)

['B' 'M' 'B' 'B' 'B' 'M' 'B' 'M' 'M' 'B' 'M' 'B' 'M' 'M' 'M' 'B' 'M' 'B'
 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'M' 'B']


In [ ]:
# Seu código aqui
print('Bs =', sum(y_pred == 'B'))
print('Ms =', sum(y_pred == 'M'))

Bs = 17
Ms = 10
